---
---

<center><h1> Project: ETL Pipeline </h1></center>


----

---

#### `Importing the required libraries`

---

In [24]:
#ETl---Lets say we have a restaurant and it has multiple sources that is generating data everyday like dbms,json,csv,xml

# and it make predictions regarding future sales or some business decision are taken.

# now the inputs are in different formats but for a decision making process we require that the data is in single format
# like in a spreadsheet or in a database,so to bridge this gap between our raw data and clean data,we have etl process.
# etl stands for extraction,transformation and load,in simple terms it prepares raw data for decision making

# extract extracts data from various sources,this extraction can either be full or partial
# transforming data involves preprocessing of data like removing null values,duplicate values and all of this happens in a
# staging area..which is a temporary place like database where all of these transformation happens
# load is the final step where we load data into target location..this loading can be of2 types, like incremental or full
# when the data is loaded after extraction and transformation for the very first time, that is a full load
# and any subsequent loading of data,for a new entry is known as a partial load.

# the mechanism trough which etl process occur is called as etl pipeline,wecan automate etl pipeline and schedule it to run
# based on business requirement

# there are multiple rulesthat fecilitate using etl pipelines like,informatica,talend,pentaho,etc.

# undersatnding the context--letssay there is an e-commerce website on which there are 100 differrent products to purchase
# in 3 different categories- which are Ac, refrigerators,and microwave,
# the data is availablein csv files and mysql tables

#product data is stored in csv file,the data change in tis file rarely.New products are addedonce in months.
# this file has 4 different colproduct id,productname,product category,product price
# next we have user table. whenever we have new user the data is stored in this table.
# in our website every 2 secs a new user is signing up and their details have been stored
# in this tablewe have unique user id,user email,user name,source from which user came like instagram,fb,etc,isprime,signuptime
# transaction table-if any product is soldin our website,then the trx details arestored here-on an avg every5-15 secs
# a trxhappens in our website,and wehave 5 different cols--transaction id,user id ,product id,transaction time,price
# our website also has a page where people can apply for a refund
# now there are some rules regarding refund--refund will be done only if the trx is done in last 48 hrs.
# the details of refund is stored in refund table,wehave ticket id,user name,transaction id,transaction amount,ticket raise time
# the company automatically do refunds in 30mis, so we need to validate the request before initiating refund.
# so for that we will create a new table called valid refund table which has-ticket id, trx id,user id,price,ticket raise time

# so we have 1 sql file and 4 mysql tables apart fromthat we have 1 more table called accounts table which has details of
# account number of customers-- but for the scope of this project lets suppose we dont have access tothat.

# understanding requirements of etl pipeline---
# signup summary gets updated every 5 mins , and we need data added in the users table in the last 5 mins to generate the results
# this is the first pipeline that we need to build.
# another requirement is we need to generate a trx summary table-in this table we will store the no.of trxs product category wise 
# and product brand wise. It will also store the total sales brand wise and product wise.This table gets updated every 10 mins
# to calc this data we will require trx tableand product data csv file.This is the second pipeline
# another requirement is we need to update valid refund table-for that we require data from 3 different sources-
# 1 refund table, we will collect last request every 30 mins, trxs- trxids generated in last 48 hours, from valid refunds
# we will check if the data is refund or not in a last 48hrs.
# so we have 2 types of reuests valid and invalid,we will put the result in csv file and we will assign the reason as well

# invalid request csv has ticket id,user name,refund reject reason
# apart fromthat we need to update the valid trx table for which the refund req is found correct

# this is the 3rd etl pipeline, and it runs every 30 mins


# now goto sql workbench---import etl_dump.sqlscript in workbench and create allthe tables--etl_dump.sql file

# then see 1.Generate User Data.ipynb-- u donot need to understand this file fully


# keep on running 1.Generate User Data.ipynb in the mean time goto workbench

#select count(*) from users; 246 rows have been entered.

# because generate user data.ipynb is running, rows will be kept on adding in users.u

#then goto 2.generate transactions data

# in product_table.csv we have product_id,product_name,product_categoryand product_price-- we are not updating anything

# in  this file. Then we have 3. generate refund table-

# now,above we have setup a simulation environment to generate realtime data.Now,before going into etl pipeline--we
# will first undrstand our pipeline--
# for extraction we have extract_users_data() will extract data oflast 5 mins fromusers table.-- this func will pass the 
# data to transform_users_data() wich will do required transformation .Finally,load_user_summary() for loading data into
# target table.All these steps will be executed in order every 5mins.
# for the 2nd pipeline ,we need to extract product data from csv file fo which define extract_product_data().. we also
# need to extract trx data in extract_transaction_data()... data from both these functions are passed into transform_trx_data()
# and finally load_transaction_summary() will load the data into required target table--- all these steps will be executed 
# every 10 mins
# For the finalpipeline, we need to extract_transaction_data(),extract_refund_data(),extract_valid_refund_data()-- we
# will use all these result and pass it into transform_refund_data() for required trnsformation and finally load_refund_table()
# togenerate invalid request csv file and also upload vald data into refund table-- Nowthis pipeline is executed every 30 mins

# for all these pipelines we will needa single DB connection

# now datain extract_product_data() changes rarely and we donot need to extract it every 10 mins. So we will use this func
# outside the pipelin

# now we will start building etl pipeline --

# etl pipeline 1 to update user summary table--


In [25]:
!pip install schedule

In [26]:
# importing the required libraries
import schedule
import time
import pandas as pd
import mysql.connector as mysql
from datetime import timedelta
import datetime

---

***Define a function to create the connection with the MySQL database. `Configure the following cell as per your system settings.`***

---

In [27]:
# connect to the database
def create_connection():
    db = mysql.connect(
    host = "localhost",
    user = "root",  ## Enter your user name here
    #password = "ABC@123", ## Enter your password here
    database = "etl", ## Enter the database name here
    #auth_plugin = "mysql_native_password",
    )

    return db

In [28]:
# now for the entirety of this project keep the generate user data,trx data,refund data files running until we r finished 
# with all the pipelines

---
---

<center><h1> ETL Pipeline 1 </h1></center>

---


![](images/pipeline-1.png)

---
---



#### `Define the Extraction Function`

---

* ***`extract_users_data()`***: It will extract the data from the `users` table within defined time range and return the dataframe.

<br>

* ***`Paramaters Required`***:
    * `Database Connection String` is required so that it can connect to the database and query the data. 
    * `Start time` and `End time` is required so that we can modify the query.

<br>

* ***`Output`***: It will return the pandas dataframe of the extracted data.

---

In [29]:
def extract_users_data(db, start_time, end_time):
    
    # create database cursor
    cursor = db.cursor()
    
    print("Extracting signup results between {} and {}".format(str(start_time),str(end_time)))
    
    # command to extract the data of last 5 minutes.
    command = "SELECT * FROM users WHERE signup_time BETWEEN '{}' AND '{}'".format(start_time, end_time)
    
    # execute the command and return the results.
    cursor.execute(command)
    data = cursor.fetchall()
    
    # return the dataframe
    return pd.DataFrame.from_records(data, columns= ['user_id',
                                                     'user_email',
                                                     'user_name',
                                                     'source',
                                                     'is_prime',
                                                     'signup_time'])

---

#### `Define the Trasformation Function`

---

* ***`transform_users_data()`***: It will use the data from the `extract_users_data()` of the last 5 minutes and do the following transformation using pandas.
    * Replace the category `Not Available` with the `Organic` in the source column.
    * Use the groupby function to calculate number of users in each category of source.
    * Use the groupby function to calculate number of prime users in each category of source.
    * Store all the results in a dictionary 
    * Add the start & end time in the dictionary and return it.
    
<br>    

* ***`Paramaters Required`***:
    * `df_user` user data dataframe of last 5 minutes. 
    * `Start time` and `End time` is required to update the output, so that we know the signup summary is between this particulae time range.

<br>

* ***`Output`***: It will return the dictionary that will be used to import the data into the transaction summary table.

---

----

In [30]:
# define the tranform function
def transform_user_data(df_user, start_time, end_time):
    
    print("Transforming User Data...")
    
    # replace the "Not Available" with the "Organic"
    df_user.source.replace("Not Available", "Organic", inplace=True)
    
    # groupby to calculate the number of users in each category of source.
    source_count = df_user.groupby(['source'])['user_id'].count()
    
    # groupby to calculate the number of prime users in each category of source.
    prime_count  = df_user.groupby(['source'])['is_prime'].sum()
    
    # create dictionary of source count
    source_count_dict = source_count.to_dict()
    
    # append prime count in the same dictionary
    for key in prime_count.to_dict():
        new_key_name = "prime_from_" + key
        source_count_dict[new_key_name] = prime_count[key]
    
    # add start_time and end_time to the dictionary 
    source_count_dict['start_time'] = str(start_time)
    source_count_dict['end_time'] = str(end_time)
    
    # return the final dictionary
    return source_count_dict

---

#### `Define the Loading Function`

---

* ***`load_users_summary()`***: It will use the results dictionary from the `transform_users_data` and load it into the `signup_summary_table`.

    
<br>    

* ***`Paramaters Required`***:

    * `result_dict` final results dictionary from the transform function. 
    * `db` database connection string to update the values in the table.

<br>

* ***`Output`***: It will not return anything.

---

In [31]:
def load_user_summary(result_dict, db):
    print("Loading User Summary Table...")
    cursor = db.cursor()
    
    # command to insert the data into the signup summary table using result dict
    command = "INSERT INTO signup_summary({col}) values{val}".format(col= ','.join(result_dict.keys()),
                                                                     val= tuple(result_dict.values()))
    cursor.execute(command)
    db.commit()

---
---

#### `Define the ETL Pipeline`

<br>

Now, we will define the pipeline function, we will take the database connection as the parameter and do the following steps.

 * Now, we will define the time for which we want to extract the data.
 * Extract the latest 5 minutes of users data.
 * Transform it to get the users signup summary.
 * Load the data into the signup_summary table.

---

In [32]:

def pipeline_to_update_user_summary(db_object):
    
    # get the current time and time before 5 minutes
    current_time = datetime.datetime.now()
    current_minus_5 = current_time - datetime.timedelta(minutes=5)
    
    print("========================================================================================")
    print("Starting ETL to update user summary!!")
    
    ## EXTRACTION
    latest_user_data = extract_users_data(db = db_object,
                                          start_time = current_minus_5,
                                          end_time=current_time)
    
    ## TRANSFORMATION
    user_summary_data = transform_user_data(df_user= latest_user_data,
                                            start_time=current_minus_5, 
                                            end_time=current_time)
    
    ## LOADING
    load_user_summary(result_dict = user_summary_data,
                      db = db_object)
    
    print("Successfully loaded the data into user summary table !!")
    print("========================================================================================")


---

<center><h1> ETL Pipeline 2 </h1></center>

---


![](images/pipeline-2.png)




---

#### `Define the extraction functions`

---

* ***`extract_products_data()`***: It will read the products data from the CSV file. 

* ***`Paramaters Required`***: CSV file path is required. It is present in the `dataset` folder.

* ***`Output`***: It will return the pandas dataframe of the CSV file.



In [33]:
# define function to extract the product_data from the CSV file
def extract_products_data(file_path):
    return pd.read_csv(file_path)

---

* ***`extract_transaction_data()`***: It will extract the data from the `transaction` table within a defined time range and return the dataframe.

* ***`Paramaters Required`***:
    * `Database Connection String` is required so that it can connect to the database and query the data. 
    * `Start time` and `End time` is required so that we can modify the query.

* ***`Output`***: It will return the pandas dataframe of the extracted data.

---

In [34]:
def extract_transaction_data(db, start_time, end_time):
    
    # create database cursor
    cursor = db.cursor()
    
    print("Extracting transactions between {} and {}".format(str(start_time),str(end_time)))
    
    # command to extract the data of last 5 minutes.
    command = "SELECT * FROM transaction WHERE transaction_time BETWEEN '{}' AND '{}'".format(start_time, end_time)
    
    # execute the command and return the results.
    cursor.execute(command)
    data = cursor.fetchall()
    
    # return the dataframe
    return pd.DataFrame.from_records(data, columns= ['transaction_id',
                                                     'user_id',
                                                     'product_id',
                                                     'transaction_time',
                                                     'price'])

---

#### `Define the Trasformation Function`

---

* ***`transform_transaction_data()`***: It will use the data from the `extract_transform_data()` of the last 10 minutes and the output of `extract_products_data()` and do the following transformation using pandas.
    * Do the left join on `transction_data` and the `products_data`.
    * Split the product_name and create a new feature `brand`.
    * Use groupby to calculate the brand-wise sales.
    * Use groupby to calculate the category-wise sales. 
    * Create dictionary of the calculated results.
    * Add start and end time to the dictionary.
    
<br>    

* ***`Paramaters Required`***:
    * `df_transaction` transaction data dataframe of last 10 minutes. 
    * `df_product` product data frame.
    * `Start time` and `End time` is required to update the output, so that we know the signup summary is between this particulae time range.

<br>

* ***`Output`***: It will return the dictionary which will be used to import the data into transaction summary.

---

----

In [35]:
# define the tranform function
def transform_transaction_data(df_transaction, df_product, start_time, end_time):
    
    print("Transforming Transaction Data...")
    
    # merge the transaction and product dataframe.
    merged_df = df_transaction.merge(df_product, how='left', on='product_id')
    
    # split the product_name to get the brand  
    merged_df['brand'] = merged_df['product_name'].apply(lambda x: x.split()[0])
    
    # calculate the brand count
    brand_count = merged_df.groupby(['brand'])['transaction_id'].count()
    
    # calculate the category count
    category_count = merged_df.groupby(['product_category'])['transaction_id'].count()
    
    # calculate the brand wise sales
    brand_wise_sales    = merged_df.groupby(['brand'])['price'].sum()
    
    # calculate the category wise sales
    category_wise_sales = merged_df.groupby(['product_category'])['price'].sum()
    
    # create dictionary 
    brand_count_dict = brand_count.to_dict()
    
    # append brand_count to dictionary
    for key in category_count.to_dict():
        brand_count_dict[key] = category_count[key]
    
    # append brand wise sales to dictionary
    for key in brand_wise_sales.to_dict():
        new_key_name = "sales_from_" + key
        brand_count_dict[new_key_name] = brand_wise_sales[key]
    
    # append category wise sales to dictionary
    for key in category_wise_sales.to_dict():
        new_key_name = "sales_from_" + key
        brand_count_dict[new_key_name] = category_wise_sales[key]
    
    # append start and end time to the dictionary
    brand_count_dict['start_time'] = str(start_time)
    brand_count_dict['end_time'] = str(end_time)
    
    return brand_count_dict

---

#### `Define the Loading Function`

---

* ***`load_transaction_summary()`***: It will use the results dictionary from the `transform_transaction_data` and load it into the `transaction_summary_table`.

    
<br>    

* ***`Paramaters Required`***:
    * `result_dict` final results dictionary from the transform function. 
    * `db` database connection string to update the values in the table.

<br>

* ***`Output`***: It will not return anything.

---

In [36]:
def load_transaction_summary(result_dict, db):
    print("Loading Transaction Summary Table...")
    cursor = db.cursor()
    
    # command to insert the data into the transaction summary using the 
    command = "INSERT INTO transaction_summary({col}) values{val}".format(col= ', '.join(result_dict.keys()),
                                                                       val= tuple(result_dict.values()))
    
    

    command = command.replace(' Air Conditioner', '`Air Conditioner`')
    command = command.replace('sales_from_Air Conditioner', '`sales_from_Air Conditioner`')
    
    cursor.execute(command)
    db.commit()

In [37]:
def pipeline_to_update_transaction_summary(db_object, products_data):
    
    current_time = datetime.datetime.now()
    current_minus_10 = current_time - datetime.timedelta(minutes=10)
    
    print("========================================================================================")
    print("Starting ETL to update transaction summary!!")
    
    ## EXTRACTION
    latest_transaction_data = extract_transaction_data(db = db_object,
                                                       start_time = current_minus_10,
                                                       end_time = current_time)
    
    ## TRANSFORMATION
    transaction_summary_data = transform_transaction_data(df_product = products_data,
                                                          df_transaction = latest_transaction_data,
                                                          start_time = current_minus_10,
                                                          end_time = current_time)
    ## LOADING
    load_transaction_summary(result_dict = transaction_summary_data,
                             db = db_object)
    
    print("Successfully loaded the data into transaction summary table !!")
    print("========================================================================================")

---
---


<center><h1> ETL Pipeline 3 </h1></center>

---


![](images/pipeline-3.png)



---
---

---

* ***`extract_refund_data()`***: It will extract the data from the `refund_detail` table within the defined time range and return the dataframe.

* ***`Paramaters Required`***:
    * `Database Connection String` is required so that it can connect to the database and query the data. 
    * `Start time` and `End time` is required so that we can modify the query.

* ***`Output`***: It will return the pandas dataframe of the CSV file.

---

In [38]:
def extract_refund_data(db, start_time, end_time):
    
    # create database cursor
    cursor = db.cursor()
    
    print("Extracting refund between {} and {}".format(str(start_time),str(end_time)))
    
    # command to extract the data of last 30 minutes.
    command = "SELECT * FROM refund_detail WHERE ticket_raise_time BETWEEN '{}' AND '{}'".format(start_time, end_time)
    
    # execute the command and return the results.
    cursor.execute(command)
    data = cursor.fetchall()
    
    # return the dataframe
    return pd.DataFrame.from_records(data, columns= ['ticket_id',
                                                     'user_name',
                                                     'transaction_id',
                                                     'transaction_amount',
                                                     'ticket_raise_time'])

---

* ***`extract_valid_refund_data()`***: It will extract only the `transaction_id` from the `valid_refund` table within defined time range and return the dataframe.

* ***`Paramaters Required`***:
    * `Database Connection String` is required so that it can connect to the database and query the data. 
    * `Start time` and `End time` is required so that we can modify the query.

* ***`Output`***: It will return the pandas dataframe of the CSV file.

---

In [39]:
def extract_valid_refund_data(db, start_time, end_time):
    
    # create database cursor
    cursor = db.cursor()
    
    print("Extracting valid refund data between {} and {}".format(str(start_time),str(end_time)))
    
    # command to extract the data of last 30 minutes.
    command = "SELECT transaction_id FROM valid_refund WHERE ticket_raise_time BETWEEN '{}' AND '{}'".format(start_time, end_time)
    
    # execute the command and return the results.
    cursor.execute(command)
    data = cursor.fetchall()
    
    # return the valid transaction IDs list
    return pd.DataFrame.from_records(data, columns= ['ticket_raise_time'])['ticket_raise_time'].to_list()


---

#### `Define the Trasformation Function`

---

* ***`transform_users_data()`***: It will use the data from the `extract_users_data()` of the last 5 minutes and do the following transformation using pandas.
    * Replace the category `Not Available` with the `Organic` in the source column.
    * Use the groupby function to calculate number of users in each category of source.
    * Use the groupby function to calculate number of prime users in each category of source.
    * Store all the results in a dictionary 
    * Add the start & end time in the dictionary and return it.
    
<br>    

* ***`Paramaters Required`***:
    * `df_user` user data dataframe of last 5 minutes. 
    * `Start time` and `End time` is required to update the output, so that we know the signup summary is between this particulae time range.

<br>

* ***`Output`***: It will return the dictionary with the user summary data.

---

----

In [40]:
def transform_refund_data(df_refund, df_transactions, refund_issued_TID):
    
    print("Transforming Refund Data...")
    
    valid_transactions = df_transactions.merge(df_refund, how='left', on='transaction_id')
    valid_transactions = valid_transactions[valid_transactions.ticket_id.isnull() == False]
    
    
    valid_transactions = valid_transactions.groupby(['transaction_id']).first().reset_index()
    
    valid_transactions_final = valid_transactions[~valid_transactions.transaction_id.isin(refund_issued_TID)]
    valid_transaction_duplicate = valid_transactions[valid_transactions.transaction_id.isin(refund_issued_TID)]
    
    
    
    valid_transactions_final = valid_transactions_final[['ticket_id', 'transaction_id', 'user_id', 'price', 'ticket_raise_time']]
    
    valid_transactions_final = [tuple((row[0],row[1],row[2],int(row[3]), str(row[4]))) for row in valid_transactions_final.to_records(index=False)]
    
    
    refund_rejected = df_refund.merge(df_transactions, how='left', on='transaction_id')
    refund_rejected['refund_reject_reason'] = None
    refund_rejected.loc[refund_rejected.product_id.isna() == True, 'refund_reject_reason'] = 'transaction_id_not_matched'
    refund_rejected.loc[refund_rejected.transaction_id.isin(refund_issued_TID), 'refund_reject_reason'] = 'already_processed'
    
    
    refund_rejected = refund_rejected[refund_rejected.refund_reject_reason.isna() == False]
    refund_rejected = refund_rejected[['ticket_id', 'user_name', 'refund_reject_reason']]
    
    
    return valid_transactions_final, refund_rejected
    

---

#### `Define the Loading Function`

---

* ***`load_users_summary()`***: It will use the results dictionary from the `transform_users_data` and load it into the `signup_summary_table`.

    
<br>    

* ***`Paramaters Required`***:
    * `result_dict` final results dictionary from the transform function. 
    * `db` database connection string to update the values in the table.

<br>

* ***`Output`***: It will not return anything.

---

In [41]:
def load_refund_data(valid_refunds, rejected_requests, db):
    print("Loading Valid Refunds Table...")
    cursor = db.cursor()
    
    command = "INSERT INTO valid_refund(ticket_id, transaction_id, user_id, price, ticket_raise_time) values(%s, %s, %s, %s, %s)"
    cursor.executemany(command, valid_refunds)
    db.commit()
    
    file_name = "rejected_request" + str(pd.datetime.now()) + ".csv"
    rejected_requests.to_csv("output_folder/" + file_name)
    
    

In [42]:
def pipeline_to_update_valid_refunds(db_object):
    
    current_time = datetime.datetime.now()
    current_minus_30 = current_time - datetime.timedelta(minutes = 30)
    current_minus_48 = current_time - datetime.timedelta(hours  = 48)
    
    print("========================================================================================")
    print("Starting ETL to update valid refunds data!!")
    
    ## EXTRACTION
    latest_refund_requests = extract_refund_data(db = db_object,
                                                 start_time = current_minus_30,
                                                 end_time = current_time)
    
    valid_transactions = extract_transaction_data(db = db_object,
                                                  start_time = current_minus_48,
                                                  end_time = current_time)
   
    refund_issued = extract_valid_refund_data(db = db_object,
                                              start_time = current_minus_48,
                                              end_time = current_time)
    
    ## TRANSFORMATION
    valid_refunds, refunds_rejected = transform_refund_data(df_refund=latest_refund_requests,
                                                            df_transactions=valid_transactions,
                                                            refund_issued_TID= refund_issued)
    
    ## LOADING
    load_refund_data(db = db_object,
                     valid_refunds = valid_refunds,
                     rejected_requests = refunds_rejected)
    
    print("Successfully loaded the data into valid refund table !!")
    print("========================================================================================")

---

<center><h1> Schedule Pipelines </h1></center>

---

![](images/pipeline-4.png)

---

In [44]:
#### Schedule Pipelines

db_object = create_connection()
products_data = pd.read_csv('dataset/product_table.csv')

schedule.every(30).seconds.do(pipeline_to_update_user_summary, db_object = db_object)
schedule.every(60).seconds.do(pipeline_to_update_transaction_summary, db_object = db_object, products_data = products_data)
schedule.every(150).seconds.do(pipeline_to_update_valid_refunds, db_object = db_object)

Every 150 seconds do pipeline_to_update_valid_refunds(db_object=<mysql.connector.connection_cext.CMySQLConnection object at 0x0000021C05D2CA00>) (last run: [never], next run: 2023-01-18 09:56:01)

In [45]:
while True:
        schedule.run_pending() 
        time.sleep(1)

Starting ETL to update user summary!!
Extracting signup results between 2023-01-18 09:49:01.863751 and 2023-01-18 09:54:01.863751
Transforming User Data...
Loading User Summary Table...
Successfully loaded the data into user summary table !!
Starting ETL to update transaction summary!!
Extracting transactions between 2023-01-18 09:44:31.784679 and 2023-01-18 09:54:31.784679
Transforming Transaction Data...
Loading Transaction Summary Table...
Successfully loaded the data into transaction summary table !!
Starting ETL to update user summary!!
Extracting signup results between 2023-01-18 09:49:33.162888 and 2023-01-18 09:54:33.162888
Transforming User Data...
Loading User Summary Table...
Successfully loaded the data into user summary table !!


KeyboardInterrupt: 

In [ ]:
#create table valid_refund(ticket_id varchar(50), transaction_id varchar(50), user_id varchar(50), price int);